<a href="https://colab.research.google.com/github/ScG4m3rLOL/ApiFestivos-Completo/blob/master/04-%20modelo%20de%20preprocesado%2C%20usando%20filtros%20de%20suma%20y%20promedio%20usando%20RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Competencia Kaggle

#Modelos y Simulación I Udea2025-2

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

Configuración de Kaggle y descarga de datos

In [ ]:
!mv kaggle.json /root/.config/kaggle/kaggle.json
!chmod 600 /root/.config/kaggle/kaggle.json

!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
!unzip -l udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

mv: cannot stat 'kaggle.json': No such file or directory
udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
  4716673  2025-09-16 01:46   submission_example.csv
 59185238  2025-09-16 01:46   test.csv
143732437  2025-09-16 01:46   train.csv
---------                     -------
207634348                     3 files


Carga de librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

Carga de datasets

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f"Train: {train.shape}")
print(f"Test: {test.shape}")

Train: (692500, 21)
Test: (296786, 20)


Identificar variable objetivo

In [ ]:
target_candidates = list(set(train.columns) - set(test.columns))
variable_objetivo = target_candidates[0]
print(f"Variable objetivo: {variable_objetivo}")

Variable objetivo: RENDIMIENTO_GLOBAL


Features

In [ ]:

train['suma_indicadores'] = train[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].sum(axis=1)
test['suma_indicadores'] = test[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].sum(axis=1)

train['promedio_indicadores'] = train[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].mean(axis=1)
test['promedio_indicadores'] = test[['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']].mean(axis=1)


categorical_cols = ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'F_ESTRATOVIVIENDA']

for col in categorical_cols:
    if col in train.columns:
        le = LabelEncoder()
        combined = pd.concat([train[col], test[col]], axis=0)
        le.fit(combined.astype(str))
        train[f'{col}_encoded'] = le.transform(train[col].astype(str))
        test[f'{col}_encoded'] = le.transform(test[col].astype(str))

Seleccion de Features

In [ ]:
exclude_cols = [variable_objetivo, 'ID']
feature_cols = [col for col in train.columns
                if col not in exclude_cols and train[col].dtype in [np.int64, np.float64]]

Preprocesamiento

In [ ]:
modelo = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

scores = cross_val_score(modelo, train[feature_cols], train[variable_objetivo], cv=5)
print(f"Cross-validation: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

Entrenamiento y evaluación

In [ ]:
modelo.fit(train[feature_cols], train[variable_objetivo])

RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200, n_jobs=-1, random_state=42)

Predicción final

In [ ]:
predicciones = modelo.predict(test[feature_cols])

Problema con la columna 'id'

Cambio de nombre de variable y descarga del submission

In [ ]:
print("Columnas en test:", test.columns.tolist())


id_candidates = [col for col in test.columns if 'id' in col.lower()]
print("Columnas que podrían ser ID:", id_candidates)


if id_candidates:
    id_real = id_candidates[0]
    print(f"Usando columna: '{id_real}'")
else:
    id_real = test.columns[0]
    print(f"Usando primera columna: '{id_real}'")


submission = pd.DataFrame({
    'ID': test[id_real],
    variable_objetivo: predicciones
})

submission.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')

Columnas en test: ['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4', 'suma_indicadores', 'promedio_indicadores', 'E_PRGM_ACADEMICO_encoded', 'E_PRGM_DEPARTAMENTO_encoded', 'F_ESTRATOVIVIENDA_encoded']
Columnas que podrían ser ID: ['ID', 'E_VALORMATRICULAUNIVERSIDAD']
Usando columna: 'ID'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>